In [54]:
using CSV
using DataFrames
using LinearAlgebra
using Plots
df = CSV.read("asset_prices.csv", DataFrame);
prices=Matrix(df);
asset_names=names(df);
p_changes = prices[2:end,:] - prices[1:end-1,:];
R = p_changes./prices[1:end-1,:];
T,n = size(R)
p_tar = 0.2/250;

In [55]:
Model = zeros((n+2,n+2));
y = zeros((n+2,1));
miu = transpose(R) * ones((T,1)) / T;
Model[1:n,1:n] = 2* transpose(R) * R;
Model[n+1,1:n] .= 1;
Model[n+2,1:n] = transpose(miu);
Model[1:n,n+1] .= 1;
Model[1:n,n+2] = miu;
y[1:n] = 2 * p_tar * T * miu;
y[n+1] = 1;
y[n+2] = p_tar;

In [56]:
Initial_Ans = (Model \ y);
W0 = Initial_Ans[1:n];
r0 = R*W0;
downsiderisk_Arr = zeros(T)
for i = 1 : T
    downsiderisk_Arr[i] = maximum([p_tar-r0[i],0])
end
W0_downsiderisk = sum(downsiderisk_Arr)/T

0.004116656070326331

In [57]:
function f(w)
    r = R*w;
    y = zeros((T,1));
    for i =1:T
        y[i] = maximum([p_tar-r[i],0])/T;
    end
    return y
end

function Df(w)
    y = f(w);
    alpha = y.>0;
    dw = zeros((T,T))
    for i = 1 : T
       dw[i,i]= alpha[i]; 
    end
    y = - dw * R
end

Df (generic function with 1 method)

In [58]:
Kmax = 300;
lambda = 1e-4;
loss_value = zeros(Kmax);
W = W0
for i =1:Kmax
    D = Df(W)
    F = f(W)
    W = W .- (transpose(D) * D + lambda * I) \ (transpose(D)* F)
    loss_value[i] = sum(F)
end

In [73]:
plot(1:Kmax, loss_value,
    xlabel = "Iteration",
    ylabel = "loss",
    title = "Downside risk vs Iterations",
    legend = false,
    grid = true)
savefig("Downside_risk.png")

In [70]:
println("Final Result:")
println("Initial downside risk is " * string(W0_downsiderisk))
println("Downside risk of portfolio optimization after " * string(Kmax) * " iteration is " * string(loss_value[Kmax]))

Final Result:
Initial downside risk is 0.004116656070326331
Downside risk of portfolio optimization after 300 iteration is 0.0036772705084149107
